In [6]:
import pandas as pd
import json

df = pd.read_csv("/home/hli962/Chunhou Project/all_comments_since_2015.csv")

# 删除空值
df = df.dropna(subset=["author_flair_text", "body"])

# 重命名
df = df[["author_flair_text", "body"]].rename(columns={
    "author_flair_text": "type",
    "body": "posts"
})

# 分批写入 JSON 数组
output_path = "posts_chunked.json"
chunk_size = 10000

with open(output_path, "w", encoding="utf-8") as f:
    f.write("[\n")
    total = len(df)
    for i in range(0, total, chunk_size):
        chunk = df.iloc[i:i+chunk_size].to_dict(orient="records")
        for j, item in enumerate(chunk):
            json.dump(item, f, ensure_ascii=False)
            # 判断是否是最后一条
            if i + j + 1 < total:
                f.write(",\n")
            else:
                f.write("\n")
    f.write("]\n")

print(f"✅ 分批导出完成，共 {total} 条，保存至：{output_path}")


FileNotFoundError: [Errno 2] No such file or directory: '/home/hli962/Chunhou Project/all_comments_since_2015.csv'

In [ ]:
#清理潘多拉数据
import os
import re
import json
import nltk
import ijson
import pandas as pd
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# --- 下载一次即可 ---
# nltk.download("punkt")
# nltk.download("stopwords")
# nltk.download("wordnet")

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r'(https?://\S+|www\.\S+)', ' ', text)
    text = text.replace('|', ' ')
    text = re.sub(r'[^a-z\s]', ' ', text)
    tokens = nltk.word_tokenize(text)
    cleaned = [
        lemmatizer.lemmatize(token)
        for token in tokens
        if token not in stop_words and len(token) > 2
    ]
    return ' '.join(cleaned)

def preprocess_large_flat_json_stream(input_path, output_dir="cleaned_batches", batch_size=1000):
    os.makedirs(output_dir, exist_ok=True)

    with open(input_path, 'r', encoding='utf-8') as f:
        items = ijson.items(f, 'item')  # ⬅ 逐条读取 JSON 数组元素

        batch = []
        batch_index = 1
        total = 0

        for item in tqdm(items, desc="📖 Streaming JSON"):
            batch.append(item)
            total += 1

            if len(batch) == batch_size:
                save_cleaned_batch(batch, output_dir, batch_index)
                batch = []
                batch_index += 1

        if batch:
            save_cleaned_batch(batch, output_dir, batch_index)

    print(f"\n✅ 清洗完成，共 {total} 条数据，分为 {batch_index} 批。")

def save_cleaned_batch(batch, output_dir, batch_index):
    df = pd.DataFrame(batch)
    tqdm.pandas(desc=f"🧹 Cleaning batch {batch_index}")
    df["posts_cleaned"] = df["posts"].astype(str).progress_apply(clean_text)

    json_path = os.path.join(output_dir, f"batch_{batch_index:03}.json")
    #csv_path = os.path.join(output_dir, f"batch_{batch_index:03}.csv")

    #df.to_csv(csv_path, index=False)
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(df.to_dict(orient="records"), f, ensure_ascii=False, indent=2)

    print(f"📁 已保存：{json_path}")


# --- 调用入口 --
if __name__ == "__main__":
    preprocess_large_flat_json_stream(
        input_path="posts_chunked.json",
        output_dir="cleaned_batches",
        batch_size=600000
    )


🧹 Cleaning batch 1: 100%|██████████| 600000/600000 [00:53<00:00, 11132.45it/s]
📖 Streaming JSON: 797578it [00:58, 10804.01it/s]  

📁 已保存：cleaned_batches/batch_001.json


🧹 Cleaning batch 2: 100%|██████████| 600000/600000 [00:55<00:00, 10846.30it/s]
📖 Streaming JSON: 1390987it [01:57, 9924.77it/s] 

📁 已保存：cleaned_batches/batch_002.json


🧹 Cleaning batch 3: 100%|██████████| 600000/600000 [00:55<00:00, 10737.25it/s]
📖 Streaming JSON: 1990829it [02:58, 9840.24it/s] 

...

📁 已保存：cleaned_batches/batch_010.json


🧹 Cleaning batch 11: 100%|██████████| 600000/600000 [00:53<00:00, 11222.80it/s]
📖 Streaming JSON: 6668904it [10:42, 4631.88it/s] 

📁 已保存：cleaned_batches/batch_011.json


📖 Streaming JSON: 6767705it [10:42, 10535.42it/s]
🧹 Cleaning batch 12: 100%|██████████| 167705/167705 [00:14<00:00, 11324.56it/s]


📁 已保存：cleaned_batches/batch_012.json

✅ 清洗完成，共 6767705 条数据，分为 12 批。


In [ ]:
import os
import json
import glob

# 设置目录和输出路径
input_dir = "cleaned_batches"
output_path = "merged_cleaned.json"

# 找到所有批次 JSON 文件（按数字顺序排序）
json_files = sorted(glob.glob(os.path.join(input_dir, "batch_*.json")))

merged_data = []
for file in json_files:
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
        merged_data.extend(data)

# 写入合并后的文件
with open(output_path, "w", encoding="utf-8") as f_out:
    json.dump(merged_data, f_out, ensure_ascii=False, indent=2)

print(f"✅ 合并完成，共 {len(merged_data)} 条，保存至：{output_path}")


✅ 合并完成，共 6835705 条，保存至：merged_cleaned.json


In [ ]:
import pandas as pd
import json

# 读取 JSON 文件
with open('merged_cleaned_pandora.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data)

# 筛选 posts_cleaned 长度 > 10 的样本
df = df[df['posts_cleaned'].str.len() > 10]
# 合法的 MBTI 类型集合
valid_types = {
    'INTJ', 'INTP', 'ENTJ', 'ENTP',
    'INFJ', 'INFP', 'ENFJ', 'ENFP',
    'ISTJ', 'ISFJ', 'ESTJ', 'ESFJ',
    'ISTP', 'ISFP', 'ESTP', 'ESFP'
}

# 只保留合法类型的样本
df = df[df['type'].isin(valid_types)]

# 然后再统计数量
type_counts = df['type'].value_counts()
print("过滤非法类型后的样本数：")
print(type_counts.sort_index())


过滤非法类型后的样本数：
type
ENFJ      6467
ENFP     48932
ENTJ     13457
ENTP     53810
ESFJ       691
ESFP      3483
ESTJ      3695
ESTP      9174
INFJ     44511
INFP     26245
INTJ    171447
INTP    220288
ISFJ      2179
ISFP      6789
ISTJ      8430
ISTP     21752
Name: count, dtype: int64


In [ ]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import nltk
from nltk.corpus import stopwords

# 确保你已经下载了停用词
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


# ✅ 1. 读取原始 JSON 文件
with open('merged_cleaned_pandora.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data)

# ✅ 2. 筛选 posts_cleaned 中有效字符（字母/汉字）个数 > 10
def count_valid_characters(text):
    return len(re.findall(r'[A-Za-z\u4e00-\u9fa5]', text))

df = df[df['posts_cleaned'].apply(count_valid_characters) > 30].copy()

# ✅ 3. 去除英文停用词
def remove_stopwords(text):
    tokens = text.split()  # 假设已经预处理过，不再分词
    filtered = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered)

df['posts_cleaned'] = df['posts_cleaned'].apply(remove_stopwords)

# ✅ 3. 保留合法的 16 种 MBTI 类型
valid_types = {
    'INTJ', 'INTP', 'ENTJ', 'ENTP',
    'INFJ', 'INFP', 'ENFJ', 'ENFP',
    'ISTJ', 'ISFJ', 'ESTJ', 'ESFJ',
    'ISTP', 'ISFP', 'ESTP', 'ESFP'
}
df = df[df['type'].isin(valid_types)].copy()

# ✅ 4. 每类最多保留 2000 条样本
df_limited = df.groupby('type').apply(
    lambda x: x.sample(n=min(2000, len(x)), random_state=42)
).reset_index(drop=True)

# ✅ 5. 拆分特征和标签
X = df_limited['posts_cleaned']
y = df_limited['type']

# ✅ 6. 拆分数据（train: 70%，val: 15%，test: 15%）
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

# ✅ 7. 还原为原格式（含 type, posts, posts_cleaned）
def reconstruct_json(X_subset, y_subset, full_df):
    df_subset = pd.DataFrame({'posts_cleaned': X_subset, 'type': y_subset})
    merged = df_subset.merge(
        full_df[['type', 'posts_cleaned', 'posts']],
        on=['type', 'posts_cleaned'],
        how='left'
    )
    return merged.to_dict(orient='records')

train_data = reconstruct_json(X_train, y_train, df_limited)
val_data = reconstruct_json(X_val, y_val, df_limited)
test_data = reconstruct_json(X_test, y_test, df_limited)

# ✅ 8. 保存为 JSON 文件（结构不变）
with open('pandora_train.json', 'w', encoding='utf-8') as f:
    json.dump(train_data, f, ensure_ascii=False, indent=2)

with open('pandora_val.json', 'w', encoding='utf-8') as f:
    json.dump(val_data, f, ensure_ascii=False, indent=2)

with open('pandora_test.json', 'w', encoding='utf-8') as f:
    json.dump(test_data, f, ensure_ascii=False, indent=2)

# ✅ 9. 完成提示
print("✅ 数据拆分完成，合法类型已筛选，并保存为 JSON 格式。")


[nltk_data] Downloading package stopwords to /home/hli962/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/tmp/ipykernel_2220419/741978598.py:43: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_limited = df.groupby('type').apply(


✅ 数据拆分完成，合法类型已筛选，并保存为 JSON 格式。


In [2]:
# -*- coding: utf-8 -*-
import pandas as pd

author_profiles_path = "author_profiles.csv"
comments_path = "all_comments_since_2015.csv"

author_profiles = pd.read_csv(author_profiles_path)

user_to_mbti = {row['author']: row['mbti'].upper() for index, row in author_profiles.iterrows() if isinstance(row['mbti'], str)}


comments = pd.read_csv(comments_path)

comments = comments[comments['author'].isin(user_to_mbti.keys())]

grouped_comments = comments.groupby('author')['body'].apply(lambda x: '|||'.join(x.dropna())).reset_index()

grouped_comments['type'] = grouped_comments['author'].map(user_to_mbti)

result = grouped_comments[['type', 'body']]
result.columns = ['type', 'posts']

output_path = "pandora_processed_comments.csv"

result.to_csv(output_path, index=False)

print("save to:", output_path)



save to: pandora_processed_comments.csv


In [4]:
import pandas as pd

# Define valid MBTI types
VALID_MBTIs = {'INTJ', 'INTP', 'INFP', 'ENTP', 'ISTP', 'ISFP', 'ESTJ', 'ISTJ', 'ESTP', 'ISFJ', 'ENFP', 'ESFP', 'ESFJ', 'ENFJ', 'INFJ', 'ENTJ'}

# Define input file path
input_path = "pandora_processed_comments.csv"

# Read data
data = pd.read_csv(input_path)

# Filter out invalid MBTI types
filtered_data = data[data['type'].isin(VALID_MBTIs)]

# To check for excluded data, use the following
invalid_data = data[~data['type'].isin(VALID_MBTIs)]
if not invalid_data.empty:
    print("The following data contains invalid MBTI types and has been excluded:")
    print(invalid_data)

# Define output file path
output_path = "filtered_processed_comments.csv"

# Write results to CSV file
filtered_data.to_csv(output_path, index=False)

print("Data has been validated and processed, saved to:", output_path)

The following data contains invalid MBTI types and has been excluded:
      type                                              posts
1180  INTX  1) Character pick doesn't matter until the reg...
1296  INTX  This is why whenever someone likes me, I usual...
1407  INTX  Eh, I tried to get into them but I couldn't. B...
2904  XNFP  When stressed I tend to withdraw and draw into...
3058  INFX  You’re definitely correct to hesitate acceptin...
3069  INFX  Yeah I've read it before I think. Maybe he wro...
3130  EXXP  And twice as many as Miggy|||Request to see th...
4382  XNTP  He should stop posting on Reddit because we ar...
4889  XNFX  YESSS. This is so pretty and magical! I wish m...
4926  INTX  I'm actually a freshman so I've only taken the...
5058  INFX  There was also [this makefile](https://github....
5751  INXJ  I beg to differ. TYPE_MENTION's can be scienti...
5798  XNTP  LOL|||I'd love to hear your arguments|||he's n...
6746  XSFP  If they have perceiving first it means they'll...


In [1]:
import json
import pandas as pd
from collections import Counter

# ✅ 修改为你的输出 JSON 文件路径
input_path = "filtered_processed_comments_300.json"

# 加载数据
with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 统计 MBTI 类型
counter = Counter(d["type"].upper() for d in data)
total = sum(counter.values())

# 构造 DataFrame
df_stats = pd.DataFrame([
    {"type": mbti_type, "count": count, "proportion": f"{count / total:.2%}"}
    for mbti_type, count in sorted(counter.items(), key=lambda x: x[1], reverse=True)
])

# 打印
print(f"\n📊 筛选后样本总数：{total}")
print(df_stats.to_string(index=False))



📊 筛选后样本总数：1261235
type  count proportion
INTP 332458     26.36%
INTJ 311165     24.67%
INFJ 119530      9.48%
INFP 111786      8.86%
ENTP 108536      8.61%
ENFP  60466      4.79%
ISTP  57552      4.56%
ENTJ  50234      3.98%
ISTJ  41010      3.25%
ENFJ  17575      1.39%
ISFP  14853      1.18%
ISFJ  11517      0.91%
ESTP   8893      0.71%
ESFP   7343      0.58%
ESTJ   4856      0.39%
ESFJ   3461      0.27%


In [1]:
import pandas as pd

# 读取 CSV 文件
csv_path = "filtered_processed_comments.csv"
df = pd.read_csv(csv_path)

# 转换为 JSON 格式（每行是一个字典）
json_path = "filtered_processed_comments.json"
df.to_json(json_path, orient="records", force_ascii=False, indent=2)

print("✅ 已保存为 JSON 文件:", json_path)

✅ 已保存为 JSON 文件: filtered_processed_comments.json
